In [3]:
!python -m jupytools export -nb "05_simple_training.ipynb" -o .

Exported: 05_simple_training.ipynb -> simple_training.py
1 notebook(s) exported into folder: .


In [2]:
%reload_ext autoreload
%autoreload 2

## Imports 

In [3]:
#export
from collections import OrderedDict
import json
import os
from os.path import dirname, join
from functools import reduce
from pdb import set_trace

import cv2 as cv
import jupytools
import jupytools.syspath
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import pretrainedmodels
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

jupytools.syspath.add(join(dirname(os.getcwd()), 'protein_project'))
jupytools.syspath.add('rxrx1-utils')
if jupytools.is_notebook():
    from tqdm import tqdm_notebook as tqdm
else:
    from tqdm import tqdm as tdqm
    
import contextlib, io
with contextlib.redirect_stderr(io.StringIO()):
    # from augmentation import Augmented, multichannel_norm
    from basedir import ROOT, NUM_CLASSES
    from catalyst_visdom import BatchMetricsPlotCallback, EpochMetricsPlotCallback
    from dataset import load_data, build_stats_index, RxRxDataset
    from model.utils import freeze_all, unfreeze_layers
    from visual import rgb, six, show_1, show

In [4]:
#export
torch.set_default_tensor_type(torch.FloatTensor)

## Dataset Preparation

### JSON

In [5]:
#export
data_dict = load_data(filenames=['train.json', 'test.json'])
train_records, test_records = data_dict['data']

### Transformations

In [6]:
#export
def bernoulli(p): return int(np.random.binomial(1, p))

In [7]:
#export
class BaseTransform:
    def __init__(self, apply_always=False, p=1.0, key='features'):
        self.apply_always = apply_always
        self.p = p
        self.key = key
    def __call__(self, x):
        if self.apply_always or bernoulli(self.p):
            return self.apply(x)
        return x
    def apply(self, x):
        raise NotImplementedError()

In [8]:
#export
class UnconditionalTransform(BaseTransform):
    def __init__(self, **kwargs):
        super().__init__(apply_always=True)

In [9]:
#export
class Compose(UnconditionalTransform):
    def __init__(self, transforms):
        super().__init__(apply_always=True)
        self.transforms = transforms
    def apply(self, dataset_record):
        return reduce(lambda x, f: f(x), self.transforms, dataset_record)

In [10]:
#export
class JoinChannels(UnconditionalTransform):
    def apply(self, record):
        record['features'] = np.stack(list(record['features'].values()))
        return record

In [11]:
#export
class Resize(UnconditionalTransform):
    def __init__(self, sz):
        super().__init__()
        self.sz = (sz, sz) if isinstance(sz, int) else sz
    def apply(self, record):
        record['features'] = cv.resize(record['features'], self.sz) 
        return record

In [12]:
#export
class ToFloat(UnconditionalTransform):
    def apply(self, record):
        record['features'] = record['features'].astype(np.float32)
        return record

In [13]:
#export
class SwapChannels(UnconditionalTransform):
    def __init__(self, new_order):
        super().__init__()
        self.new_order = new_order
    def apply(self, record):
        record['features'] = record['features'].transpose(*self.new_order)
        return record

In [14]:
#export
class VerticalFlip(BaseTransform):
    def apply(self, record): 
        record['features'] = cv.flip(record['features'], 0)
        return record

In [15]:
#export
class HorizontalFlip(BaseTransform):
    def apply(self, record): 
        record['features'] = cv.flip(record['features'], 1)
        return record

In [16]:
#export
class PixelStatsNorm(UnconditionalTransform):
    def __init__(self, stats):
        super().__init__()
        self.stats = stats
    def apply(self, record):
        keys = [(record['id_code'], i+1, record['site']) for i in range(6)]
        mean = np.stack([self.stats[key]['mean'] for key in keys]).astype(np.float32)
        std = np.stack([self.stats[key]['std'] for key in keys]).astype(np.float32)
        mean, std = [arr.reshape(len(arr), 1, 1) for arr in (mean, std)]
        norm = (record['features'] - mean)/(std + 1e-8)
        record['features'] = norm
        return record

In [17]:
#export
class Augmented(Dataset):
    def __init__(self, ds, tr):
        self.ds = ds
        self.tr = tr
    def __len__(self):
        return len(self.ds)
    def __getitem__(self, index):
        return self.tr(self.ds[index])

### Composition and Normalization Stats Reading

In [18]:
#export
stats = build_stats_index(ROOT/'pixel_stats.csv')

train_transform = Compose([
    JoinChannels(),
    SwapChannels((1, 2, 0)),
    Resize(224),
    ToFloat(),
    VerticalFlip(p=0.25),
    HorizontalFlip(p=0.25),
    SwapChannels((2, 0, 1)),
    PixelStatsNorm(stats)
])

test_transform = Compose([
    JoinChannels(),
    SwapChannels((1, 2, 0)),
    Resize(224),
    ToFloat(),
    SwapChannels((2, 0, 1)),
    PixelStatsNorm(stats)
])

In [19]:
list(stats)[:5]

[('HEPG2-01_1_B02', 1, 1),
 ('HEPG2-01_1_B02', 2, 1),
 ('HEPG2-01_1_B02', 3, 1),
 ('HEPG2-01_1_B02', 4, 1),
 ('HEPG2-01_1_B02', 5, 1)]

## Pipeline

### Split

In [20]:
#export
def split(records):
    experiments = np.unique([r['experiment'] for r in records])
    cell_types = np.unique([r['cell_type'] for r in records])
    holdouts = []
    for ct in cell_types:
        holdout = np.random.choice(experiments[
            [exp.startswith(ct) for exp in experiments]])
        holdouts.append(holdout)

    train_exp, valid_exp = [exp for exp in experiments if exp not in holdouts], holdouts
    print(f'Train experiments: {train_exp}')
    print(f'Valid experiments: {valid_exp}')
    train = [r for r in records if r['experiment'] in train_exp]
    valid = [r for r in records if r['experiment'] in valid_exp]
    print(f'Train size: {len(train)}, valid size: {len(valid)}')
    
    return train, valid

### Loaders

In [21]:
#export
bs = 650
n_cpu = 12

train_records, valid_records = split(train_records)

trn_ds = RxRxDataset(train_records, num_classes=NUM_CLASSES)
aug_trn_ds = Augmented(trn_ds, train_transform)

val_ds = RxRxDataset(valid_records, num_classes=NUM_CLASSES)
aug_val_ds = Augmented(val_ds, train_transform)

tst_ds = RxRxDataset(test_records, num_classes=NUM_CLASSES)
aug_tst_ds = Augmented(tst_ds, test_transform)

loaders = OrderedDict([
    ('train', DataLoader(aug_trn_ds, batch_size=bs, shuffle=True, num_workers=n_cpu)),
    ('valid', DataLoader(aug_val_ds, batch_size=bs, shuffle=False, num_workers=n_cpu)),
])

test_loader = DataLoader(aug_tst_ds, batch_size=bs, shuffle=False, num_workers=n_cpu)

Train experiments: ['HEPG2-02', 'HEPG2-03', 'HEPG2-04', 'HEPG2-05', 'HEPG2-06', 'HEPG2-07', 'HUVEC-01', 'HUVEC-02', 'HUVEC-03', 'HUVEC-04', 'HUVEC-06', 'HUVEC-07', 'HUVEC-08', 'HUVEC-09', 'HUVEC-10', 'HUVEC-11', 'HUVEC-12', 'HUVEC-13', 'HUVEC-14', 'HUVEC-15', 'HUVEC-16', 'RPE-01', 'RPE-02', 'RPE-04', 'RPE-05', 'RPE-06', 'RPE-07', 'U2OS-02', 'U2OS-03']
Valid experiments: ['HEPG2-01', 'HUVEC-05', 'RPE-03', 'U2OS-01']
Train size: 64170, valid size: 8860


## Model

In [22]:
#export
from catalyst.contrib.modules import GlobalConcatPool2d

In [23]:
#export
def create_model(model_name):
    return pretrainedmodels.__dict__[model_name]()

In [24]:
#export
class DoubleModel(nn.Module):
    
    def __init__(self, model_name, num_classes):
        super().__init__()
        self.m1 = create_model(model_name)
        self.m2 = create_model(model_name)
        self.pool = GlobalConcatPool2d()
        
        out_features = self.m1.last_linear.in_features * 2
        
        self.top = nn.Sequential(
            nn.Linear(out_features * 2, out_features),
            nn.LeakyReLU(),
            nn.Dropout(),
            nn.Linear(out_features, num_classes)
        )
        
    def forward(self, x):
        t1 = self.pool(self.m1.features(x[:, 0:3, :]))
        t2 = self.pool(self.m2.features(x[:, 3:6, :]))
        feat = torch.cat([t1, t2], dim=1)
        logits = self.top(feat.squeeze())
        return logits

In [25]:
# class SixChannelModel(nn.Module):
    
#     def __init__(self, model_name, num_classes):
#         super().__init__()
#         model = create_model(model_name)
#         feat_dim = model.last_linear.in_features
#         model.last_linear = nn.Sequential(
#             nn.Linear(feat_dim, feat_dim),
#             nn.LeakyReLU(negative_slope=0.01),
#             nn.Dropout(0.25),
#             nn.Linear(feat_dim, num_classes)
#         )
#         conv0 = model.features.conv0
#         new_conv = nn.Conv2d(6, 64, 7, 2, 3, bias=False)
#         new_conv.weight.data[:,0:3,:] = conv0.weight.data.clone()
#         new_conv.weight.data[:,3:6,:] = conv0.weight.data.clone()
#         model.features.conv0 = new_conv
#         del conv0
#         self.model = model
    
#     def forward(self, x):
#         return self.model(x)

In [26]:
# model = SixChannelModel('densenet121', NUM_CLASSES)
# freeze_all(model)
# unfreeze_layers(model, ['model.last_linear'])

In [27]:
#export
model = DoubleModel('densenet121', NUM_CLASSES)
freeze_all(model)
unfreeze_layers(model, ['top'])

Unfreezing layer top


In [28]:
#export
device = torch.device('cuda:1')

In [29]:
#export
loss_fn = nn.CrossEntropyLoss()

In [30]:
#export
class RollingLoss:
    def __init__(self, smooth=0.98):
        self.smooth = smooth
        self.prev = 0
    def __call__(self, curr, batch_no):
        a = self.smooth
        avg_loss = a*self.prev + (1 - a)*curr
        debias_loss = avg_loss/(1 - a**batch_no)
        self.prev = avg_loss
        return debias_loss

In [31]:
# from catalyst.contrib.schedulers import OneCycleLR
# from catalyst.data.dataset import ListDataset
# from catalyst.dl.callbacks import AccuracyCallback, AUCCallback, F1ScoreCallback
# from catalyst.dl.callbacks import EarlyStoppingCallback
# from catalyst.dl.runner import SupervisedRunner
# from catalyst.utils import get_one_hot

In [32]:
# os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [33]:
# opt = torch.optim.SGD(params=[
#     dict(params=model.model.last_linear.parameters(), lr=0.001)
# ])
# sched = torch.optim.lr_scheduler.ReduceLROnPlateau(opt)
# runner = SupervisedRunner()
# epochs = 1000

In [34]:
# #export
# runner.train(
#     model=model,
#     num_epochs=epochs,
#     criterion=loss_fn,
#     optimizer=opt,
#     scheduler=sched,
#     logdir='/home/ck/logs/protein/catalyst',
#     loaders=loaders,
#     callbacks=[
#         AccuracyCallback(num_classes=NUM_CLASSES, accuracy_args=[1]),
#         EarlyStoppingCallback(patience=10, minimize=False, metric='accuracy01'),
#         BatchMetricsPlotCallback(use_env_creds=True),
#         EpochMetricsPlotCallback(use_env_creds=True)
#     ],
#     verbose=True
# )

In [35]:
#export
from visdom import Visdom

epochs = 1000
patience = 10

opt = torch.optim.SGD(params=[
    dict(params=model.top.parameters(), lr=0.001)
])

sched = torch.optim.lr_scheduler.ReduceLROnPlateau(opt, patience=10, mode='max')
model = model.to(device)
rolling_loss = dict(train=RollingLoss(), valid=RollingLoss())
steps = dict(train=0, valid=0)

trials = 0
best_metric = -np.inf
history = []
stop = False

vis = Visdom(server='0.0.0.0', port=9090,
             username=os.environ['VISDOM_USERNAME'],
             password=os.environ['VISDOM_PASSWORD'])

for epoch in range(1, epochs+1):
    print(f'Epoch [{epoch}/{epochs}]')
    iteration = dict(epoch=epoch, train_loss=list(), valid_loss=list())
    
    for name, loader in loaders.items():
        is_training = name == 'train'
        count = 0
        metric = 0.0
        
        with torch.set_grad_enabled(is_training):
            for batch in loader:
                steps[name] += 1
                opt.zero_grad()
                x = batch['features'].to(device)
                y = batch['targets'].to(device)
                out = model(x)
                
                if is_training:
                    loss = loss_fn(out, y)
                    loss.backward()
                    opt.step()
                
                avg_loss = rolling_loss[name](loss.item(), steps[name])
                iteration[f'{name}_loss'].append(avg_loss)
                y_pred = out.softmax(dim=1).argmax(dim=1)
                acc = (y_pred == y).float().mean().item()
                metric += acc
                count += len(batch)
                vis.line(X=[steps[name]], Y=[avg_loss], name=f'{name}_loss', 
                         win=f'{name}_loss', update='append', 
                         opts=dict(title=f'Running Loss [{name}]'))
        
        metric /= count
        sched.step(metric)
        iteration[f'{name}_acc'] = metric
        vis.line(X=[epoch], Y=[avg_loss], name=f'{name}', win='avg_loss',
                 update='append', opts=dict(title='Average Epoch Loss'))
        vis.line(X=[epoch], Y=[metric], name=f'{name}', win='accuracy', 
                 update='append', opts=dict(title=f'Accuracy'))
        
        if not is_training:
            if metric > best_metric:
                trials = 0
                best_metric = metric
                print('Score improved!')
                torch.save(model.state_dict(), f'train.{epoch}.pth')
            else:
                trials += 1
                if trials >= patience:
                    stop = True
                    break
        
        last_loss = iteration[f'{name}_loss'][-1]
        
        print(f'{name} metrics: accuracy={metric:2.3%}, loss={last_loss:.4f}')
    
    history.append(iteration)
    
    print('-' * 80)
    
    if stop:
        print(f'Early stopping on epoch: {epoch}')
        break

torch.save(history, 'history.pth')

W0914 12:13:55.860801 140442564388672 __init__.py:505] Setting up a new session...


Epoch [1/1000]
train metrics: accuracy=0.008%, loss=7.2169
Score improved!
valid metrics: accuracy=0.003%, loss=7.1448
--------------------------------------------------------------------------------
Epoch [2/1000]


Process Process-28:
Process Process-25:
Process Process-35:
Process Process-33:
Process Process-34:
Process Process-26:
Process Process-36:
Process Process-27:
Process Process-31:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/ck/anaconda3/envs/fastai_10/lib/python3.7/multiprocessing/process.py", line 300, in _bootstrap
    util._exit_function()
  File "/home/ck/anaconda3/envs/fastai_10/lib/python3.7/multiprocessing/process.py", line 300, in _bootstrap
    util._exit_function()
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/ck/anaconda3/envs/fastai_10/lib/python3.7/multiprocessing/util.py", line 325, in _exit_function
    _run_finalizers()
  File "/home/ck/anaconda3/envs/fastai_10/lib/python3.7/multiprocessing/util.py", line 325, in _exit_fu

  File "/home/ck/anaconda3/envs/fastai_10/lib/python3.7/threading.py", line 1048, in _wait_for_tstate_lock
    elif lock.acquire(block, timeout):
KeyboardInterrupt
  File "/home/ck/anaconda3/envs/fastai_10/lib/python3.7/threading.py", line 1048, in _wait_for_tstate_lock
    elif lock.acquire(block, timeout):
  File "/home/ck/anaconda3/envs/fastai_10/lib/python3.7/threading.py", line 1048, in _wait_for_tstate_lock
    elif lock.acquire(block, timeout):
  File "/home/ck/anaconda3/envs/fastai_10/lib/python3.7/threading.py", line 1048, in _wait_for_tstate_lock
    elif lock.acquire(block, timeout):
  File "/home/ck/anaconda3/envs/fastai_10/lib/python3.7/threading.py", line 1048, in _wait_for_tstate_lock
    elif lock.acquire(block, timeout):
  File "/home/ck/anaconda3/envs/fastai_10/lib/python3.7/threading.py", line 1048, in _wait_for_tstate_lock
    elif lock.acquire(block, timeout):
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrup

KeyboardInterrupt: 

In [ ]:
# history

In [ ]:
#!rm -rf *.pth